# Nayak, Anil Kumar
# 1001-396-015
# 2017-03-25
# Project_Report_02_02

In [1]:
import cv2
import imutils
import numpy as np
from imutils.object_detection import non_max_suppression
import cv2.ml as svmlib
import cv2
import os



# Read Videos

In [2]:
def read_video_details(train_base_path):
    training_class_path = os.listdir(train_base_path)
    traing_data_details = []

    for class_path in training_class_path:
        details = {}
        details['class'] = class_path
        details['path'] = train_base_path +"\\"+ class_path
        videos_names = os.listdir(train_base_path +"\\"+ class_path)
        details['videos']  = videos_names
        traing_data_details.append(details)

    return traing_data_details

def prepare_data_for_clustering(training_data,training_class,flag,number_of_features):
    training_matrix_data = []
    training_matrix_class = []

    for index in range(0,np.shape(training_data)[0]):
        frames = training_data[index]

        if flag == 1:
            if np.size(frames) > 18:
                row = []
                for frame in frames:
                    frame_array = np.asarray(frame,dtype=int)
                    row = np.append(row,frame_array)
                feature = np.size(row)

                if number_of_features < feature:
                    number_of_features = feature
                training_matrix_data.append(row)
                training_matrix_class.append(training_class[index])
        else:
            row = []
            for frame in frames:
                frame_array = np.asarray(frame, dtype=int)
                row = np.append(row, frame_array)

            training_matrix_data.append(row)
            training_matrix_class.append(training_class[index])

    matrix = np.zeros((np.shape(training_matrix_data)[0],number_of_features), dtype=np.float32)
    for index in range(0,np.shape(training_matrix_data)[0]):
        row = training_matrix_data[index]
        count = 0
        for element in row:
            matrix[index][count] = element
            count = count + 1
        # row_size = np.size(row)
        # required_zeros = number_of_features-row_size
        # if required_zeros > 0:

    return matrix,np.asarray(training_matrix_class),number_of_features


def get_class_label(class_dtl,class_definition):
    return class_definition[class_dtl]

# Detect Feature in Videos

In [7]:
def feature_detection(traing_data_details,class_definition):
    training_data = []
    training_class = []
    for training in traing_data_details:
        class_dtl = training['class']
        video_path = training['path']
        videos = training['videos']

        for video in videos:

            # cv2.namedWindow("preview")
            cam = cv2.VideoCapture(video_path + "\\" + video)
            hog = cv2.HOGDescriptor()
            hog.setSVMDetector(cv2.HOGDescriptor_getDefaultPeopleDetector())

            try:
                all_frame_in_one_video = []
                while True:
                    _, image1 = cam.read()
                    image = imutils.resize(image1, width=min(400, image1.shape[1]))
                    orig = image.copy()

                    # detect people in the image
                    (rects, weights) = hog.detectMultiScale(image, winStride=(2, 2), padding=(8, 8), scale=1.05)
                    # print(rects)
                    # draw the original bounding boxes
                    for (x, y, w, h) in rects:
                        y = y + 10
                        pad_w, pad_h = int(0.2 * w), int(0.2 * h)

                        top_x = x + pad_w + int(w / 2)
                        top_y = y + pad_h

                        mid_x = x + pad_w + int(w / 2)
                        mid_y = y + pad_h + int((h - pad_h) / 2)

                        bottom_x = x + pad_w + int(w / 2)
                        bottom_y = y + pad_h + h - pad_h

                        one_frame_detail = [top_x, top_y, mid_x, mid_y, bottom_x, bottom_y]
                        all_frame_in_one_video.append(one_frame_detail)

                        cv2.rectangle(orig, (x + pad_w, y + pad_h), (x + w - pad_w, y + h - pad_h), (0, 255, 0), 1)

                    # apply non-maxima suppression to the bounding boxes using a
                    # fairly large overlap threshold to try to maintain overlapping
                    # boxes that are still people
                    rects = np.array([[x, y, x + w, y + h] for (x, y, w, h) in rects])
                    pick = non_max_suppression(rects, probs=None, overlapThresh=0.65)

                    # draw the final bounding boxes
                    # for (xA, yA, xB, yB) in pick:
                    #    cv2.rectangle(image, (xA, yA), (xB, yB), (0, 255, 0), 2)

                    # show some information on the number of bounding boxes
                    # filename = imagePath[imagePath.rfind("/") + 1:]
                    # print("[INFO] {}: {} original boxes, {} after suppression".format(
                    #   filename, len(rects), len(pick)))


                    # cv2.imshow("preview", orig)

                    #key = cv2.waitKey(10)
                    #if key == 27:
                    #    cv2.destroyWindow("preview")
                    #    break

            except:
                # print(np.size(all_frame_in_one_video))
                if np.size(all_frame_in_one_video) != 0:
                    training_data.append(all_frame_in_one_video)
                    training_class.append(get_class_label(class_dtl, class_definition))

            cam.release()
            # cv2.destroyWindow("preview")

    return training_data,training_class

# Train SVM

In [4]:
def train_svm(trainData,responses):
    svm_params = dict( kernel_type = svmlib.SVM_LINEAR, svm_type=svmlib.SVM_C_SVC, C=2.67, gamma=5.383)

    svm = cv2.ml.SVM_create()
    svm.train(trainData,cv2.ml.ROW_SAMPLE,responses )#p=svm_paramaramss
    svm.save('svm_data.dat')
    return svm

def predict(testData):
    svm = cv2.ml.SVM_create()
    svm.load('svm_data.dat')
    result = svm.predict(testData)
    return result

def predict_multiple_data(testData,svm):
    results = svm.predict(testData)
    return results

# Test SVM

In [5]:
def predict_data_all(test_base_path,class_definition,number_of_features,svm):
    testing_data_details = read_video_details(test_base_path)

    # Find Feature in Test Data
    testing_data, testing_class = feature_detection(testing_data_details, class_definition)

    # Prepare Testing Data for prediction
    flag = 0
    testing_matrix_data, testing_matrix_class,number_of_features = prepare_data_for_clustering(testing_data, testing_class, flag,number_of_features)

    # Predict Class for Testing Data
    result = predict_multiple_data(testing_matrix_data,svm)

    return result


# Pose Estimation

In [9]:
train_base_path = "train"
class_definition = {'jump': 1, 'run': 2,'pjump': 3,'skip': 4,'walk': 5}

#Fetching Videos for Training
# train_base_path is outer folder where all the folder resides
# Ex
# train
#   -walk <forlder name is for class label for video>
#       -Videos <training videos>
#   -jump
#       -Videos
training_data_details = read_video_details(train_base_path)

#Detect Human and their skeleton and find the feature for classification
training_data,training_class = feature_detection(training_data_details,class_definition)

#Preparing data for classification
flag = 1
number_of_features = 0
training_matrix_data,training_matrix_class,number_of_features = prepare_data_for_clustering(training_data,training_class,flag,number_of_features)
training_matrix_data = np.float32(training_matrix_data)

#Training Multiple Class SVM Classifier
svm = train_svm(training_matrix_data,training_matrix_class)


#Testing Data
# test
#   -walk <forlder name is for class label for video>
#       -Videos <testing videos>
#   -jump
#       -Videos
test_base_path = "test"
#class_definition = {'jump': 1, 'run': 2,'pjump': 3,'skip': 4,'walk': 5}
result = predict_data_all(test_base_path,class_definition,number_of_features,svm)

print(result[1])

[[ 2.]]
